In [30]:
%profile "cloudbend"

%glue_version "3.0"
%iam_role "arn:aws:iam::898546127587:role/GlueSessions"
%idle_timeout 60

%worker_type "G.1X"
%number_of_workers 10

%additional_python_modules tqdm

Previous profile: cloudbend
Setting new profile to: cloudbend
Setting Glue version to: 3.0
Current iam_role is arn:aws:iam::898546127587:role/GlueSessions
iam_role has been set to arn:aws:iam::898546127587:role/GlueSessions.
Current idle_timeout is 60 minutes.
idle_timeout has been set to 60 minutes.
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 10
Additional python modules to be included:
tqdm


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

Authenticating with profile=cloudbend
glue_role_arn defined by user: arn:aws:iam::898546127587:role/GlueSessions
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 10
Session ID: f0f45170-a03b-4b13-a8fa-8c691624be1c
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
--additional-python-modules tqdm
Waiting for session f0f45170-a03b-4b13-a8fa-8c691624be1c to get into ready status...
Session f0f45170-a03b-4b13-a8fa-8c691624be1c has been created.



In [5]:
import boto3

s3 = boto3.client("s3")

# list all files
response = s3.list_objects_v2(Bucket="nyc-tlc", Prefix="trip data/")
paths = [f"s3://nyc-tlc/{x['Key']}" for x in response["Contents"]]

# only include parquet files
paths = [x for x in paths if x.endswith(".parquet")]

len(paths)

431


In [6]:
from pyspark.sql.functions import col, lit

bucket = "pyicebergbenchmark-icebergbucket89dd3fa6-1qhkzajoxgpzo"

column_dtypes = {
    "PUlocationID": "string",
    "DOlocationID": "string",
    "trip_type": "string",
    "rate_code": "string",
    "improvement_surcharge": "double",
    "congestion_surcharge": "double",
    "ehail_fee": "double",
    "VendorID": "string",
    "SR_Flag": "string",
    "airport_fee": "double",
    "payment_type": "string",
}

def standardize_fragment(path: str, prefix: str) -> int:
    df = spark.read.parquet(path)

    for column, dtype in column_dtypes.items():
        if column in df.columns:
            df = df.withColumn(column, col(column).cast(dtype))
        else:
            df = df.withColumn(column, lit(None).cast(dtype))

    total = df.count()

    df.write.mode("overwrite").parquet(f"s3://{bucket}/raw/{prefix}/")

    return total

In [7]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as executor:
    totals = executor.map(
        lambda args: standardize_fragment(*args),
        [(path, str(i)) for i, path in enumerate(paths)]
    )

    sum(totals)

3318228997


In [8]:
df = (
    spark.read
        .option("mergeSchema", "true")
        .parquet(f"s3://{bucket}/raw/*/")
)

df.printSchema()

Py4JJavaError: An error occurred while calling o21629.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 16 in stage 2158.0 failed 4 times, most recent failure: Lost task 16.3 in stage 2158.0 (TID 18098) (172.35.220.134 executor 1): org.apache.spark.SparkException: Failed merging schema:
root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- VendorID: string (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- payment_type: string (nu

In [9]:
df.show()

NameError: name 'df' is not defined


In [34]:
%stop_session

Stopping session: 35b91180-4f42-452d-95de-b6fcfce2734a


Stopped session.
